In [17]:
import copy
import numpy as np
from persor import BVHparser
from scipy.spatial.transform import Rotation

import matplotlib.pyplot as plt
from matplotlib.animation import ArtistAnimation

In [18]:
def get_joint_coords(skeleton, joint, coords):
    offset = skeleton[joint]["offset"]
    parent_joint = skeleton[joint]["joint"]
    child_joints = skeleton[joint]["children"]

    if parent_joint == None:
        return

    parent_coord = coords[parent_joint]["coord"]

    current_coord = [
        parent_coord[0] + offset[0],
        parent_coord[1] + offset[1],
        parent_coord[2] + offset[2],
    ]
    coords[joint] = {"coord": current_coord, "parent": parent_joint}

    for child in child_joints:
        get_joint_coords(skeleton, child, coords)


def skelton2coords(skeleton):
    coords = {"root": {"coord": skeleton["root"]["offset"], "parent": None}}
    skeleton["root"]

    child_joints = skeleton["root"]["children"]
    for child in child_joints:
        get_joint_coords(skeleton, child, coords)

    get_joint_coords(skeleton, "root", coords)

    return coords


def rotated_offset(skeleton, frame):
    skeleton_copy = copy.deepcopy(skeleton)
    for joint, data in skeleton_copy.items():
        if joint == "root" or joint.startswith("_"):
            continue

        x_rotate = np.deg2rad(frame[f"{joint}_Xrotation"])
        y_rotate = np.deg2rad(frame[f"{joint}_Yrotation"])
        z_rotate = np.deg2rad(frame[f"{joint}_Zrotation"])

        rot = Rotation.from_rotvec(np.array([y_rotate, x_rotate, z_rotate]))
        skeleton_copy[joint]["offset"] = rot.apply(data["offset"])

    return skeleton_copy


def convert2coordDict(bvhp):
    df = bvhp.get_motion_df()
    skeleton = bvhp.get_skeleton()

    data = []
    for i in range(len(df)):
        skeleton_rotated = rotated_offset(skeleton, df.iloc[i])
        coords = skelton2coords(skeleton_rotated)
        data.append(coords)

    return np.array(data)

In [19]:
bvhp = BVHparser("./data/bvh/体を曲げる運動.bvh")
# bvhp = BVHparser("./data/bvh/両手を上げ下げ.bvh")
coords = convert2coordDict(bvhp)


In [20]:
fig = plt.figure(figsize=(5, 5))

frames = []
for coord in coords[270:3000:10]:
    frame = [
        plt.plot(c["coord"][0], c["coord"][1], marker="o", color="blue")[0]
        for c in coord.values()
    ]
    frames.append(frame)

ani = ArtistAnimation(fig, frames, interval=1)

ani.save("animation.gif", writer="pillow")
plt.close()